In [1]:
import os
import numpy as np
from natsort import natsorted
import glob
import shutil
import cv2
import skimage.io
from skimage import exposure
import nibabel as nib

import nibabel
import nibabel.affines
import numpy
import scipy.ndimage

/home/tmquan/anaconda3/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
def resample_nii(image, pixel_size, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    # print(scan[0].SliceThickness, scan[0].PixelSpacing)
    spacing = list(pixel_size) #np.array(list([scan[0].SliceThickness]) + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = np.array([old/new for (old, new) in zip(spacing, new_spacing)])
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, order=3)
    
    return image, new_spacing

In [3]:
import pydrr
import pydrr.autoinit
import SimpleITK as sitk
import matplotlib.pyplot as plt
import mpl_toolkits.axes_grid1
import numpy as np
import sys
from pydrr import utils

In [4]:
def volume2image(volume, spacing=[1,1,1]):
    print(pydrr.get_supported_kernels())
    print(pydrr.get_current_kernel())
    pydrr.set_current_kernel('render_with_cubic_interp')
    print(pydrr.get_current_kernel())

    # volume = pydrr.utils.HU2Myu(volume - 600, 0.2683)
    # https://vincentblog.xyz/posts/medical-images-in-python-computed-tomography
    #volume = pydrr.utils.window_image(volume, 200, 800)
    #volume = pydrr.utils.window_image(volume, 400, 400)
    volume = pydrr.utils.HU2Myu(volume - 50, 0.2683)

    pm_Nx3x4, image_size, image_spacing = load_test_projection_matrix(
        SDD=2048, 
        SOD=2048, 
        image_size=[1280, 1280], 
#         spacing=[1, 1]
        )
    T_Nx4x4 = load_test_transform_matrix()

    # Construct objects
    volume_context = pydrr.VolumeContext(volume.astype(np.float32), spacing)
    geometry_context = pydrr.GeometryContext()
    geometry_context.projection_matrix = pm_Nx3x4

    n_channels = T_Nx4x4.shape[0] * pm_Nx3x4.shape[0]
    detector = pydrr.Detector(pydrr.Detector.make_detector_size(image_size, n_channels), image_spacing)
    # detector = pydrr.Detector.from_geometry(geometry_context, T_Nx4x4) # You can use from_geometry if you set pixel_size and image_size.
    projector = pydrr.Projector(detector, 1.0).to_gpu()

    # Host memory -> (Device memory) -> Texture memory
    t_volume_context = volume_context.to_texture()

    d_image = projector.project(t_volume_context, geometry_context, T_Nx4x4)

    # Device memory -> Host memory
    image = d_image.get()
    return image
#     image = d_image.get()
#     print('Result image shape:', image.shape)
#     plt.figure(figsize=(16,9))
#     n_show_channels = 3
#     for i in range(min(image.shape[2], n_show_channels)):
#         ax = plt.subplot(1, min(image.shape[2], n_show_channels), i+1)
#         divider = mpl_toolkits.axes_grid1.make_axes_locatable(ax)
#         cax = divider.append_axes('right', '5%', pad='3%')
#         im = ax.imshow(image[:, :, i], interpolation='none', cmap='gray')
#         plt.colorbar(im, cax=cax)
#     plt.show()

#     save_image('drr.mhd', image, image_spacing)


def load_test_projection_matrix(SDD=2000, SOD=1800, image_size=[1280, 1280], spacing=[0.287, 0.287] ):

    if isinstance(image_size, list):
        image_size = np.array(image_size)

    if isinstance(spacing, list):
        spacing = np.array(spacing)

    extrinsic_R = utils.convertTransRotTo4x4([[0,0,0,90,0,0],
                                              [0,0,0,0,90,0],
                                              [0,0,0,0,0,90]])

    #print('extrinsic_R:', extrinsic_R)
    #print('extrinsic_R.shape:', extrinsic_R.shape)

    extrinsic_T = utils.convertTransRotTo4x4([0,0,-SOD,0,0,0])

    #print('extrinsic_T:', extrinsic_T)
    #print('extrinsic_T.shape:', extrinsic_T.shape)


    extrinsic = utils.concatenate4x4(extrinsic_T, extrinsic_R)

    #print('extrinsic:', extrinsic)
    #print('extrinsic.shape:', extrinsic.shape)


    intrinsic = np.array([[-SDD/spacing[0], 0, image_size[0]/2.0], # unit: [pixel]
                          [0, -SDD/spacing[1], image_size[1]/2.0],
                          [0,                0,               1]])

    #print('intrinsic:', intrinsic)
    #print('intrinsic.shape:', intrinsic.shape)


    pm_Nx3x4 = utils.constructProjectionMatrix(intrinsic, extrinsic)
    #pm_Nx3x4 = np.repeat(pm_Nx3x4, 400, axis=0)

    #print('pm_Nx3x4:', pm_Nx3x4)
    #print('pm_Nx3x4.shape:', pm_Nx3x4.shape)

    return pm_Nx3x4, image_size, spacing

def load_test_transform_matrix(n_channels=1):
    T_Nx6 = np.array([0,0,0,90,0,0])
    T_Nx6 = np.expand_dims(T_Nx6, axis=0)
    T_Nx6 = np.repeat(T_Nx6, n_channels, axis=0)
    T_Nx4x4 = utils.convertTransRotTo4x4(T_Nx6)

    #print('T_Nx4x4:', T_Nx4x4)
    #print('T_Nx4x4.shape:', T_Nx4x4.shape)

    return T_Nx4x4

In [ ]:
folder = '/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/'
output = 'tmp_mosmed'
shutil.rmtree(output, ignore_errors=True)
os.makedirs(output)

filenames = natsorted(glob.glob(os.path.join(folder, '**/*nii.gz')))

viz = False
for idx, filename in enumerate(filenames):
    print('='*50)
    #img = nib.load(filename)
    print(filename)    
    nib_data = nib.load(filename)
    print(nib_data.header.get_zooms())
    np_data_resample, _ = resample_nii(nib_data.get_fdata(), nib_data.header.get_zooms())
    volume = np_data_resample.transpose(2, 1, 0)[:,::-1,:]
    print(nib_data.get_fdata().shape, np_data_resample.shape)
    
    np_xr2 = volume2image(np_data_resample)
    
    filename = os.path.basename(filename).split('.')[0]
    
    # Save image
    image = np.rot90(np_xr2[:,::-1,2], 2)
    image = cv2.resize(image, (256, 256))
    if viz:
        plt.imshow(image, interpolation='none', cmap='gray')
        plt.show()
    skimage.io.imsave(os.path.join(output, 'xr2_' + filename + '.png'), image)
    
    # Save volume
    factor = [64/volume.shape[0], 256/volume.shape[1], 256/volume.shape[2]]
    volume = scipy.ndimage.zoom(volume, factor, order=3) # Bi-cubic
    if viz:
        plt.imshow(volume[0,:,:], interpolation='none', cmap='gray')
        plt.show()
        plt.imshow(volume[-1,:,:], interpolation='none', cmap='gray')
        plt.show()
    skimage.io.imsave(os.path.join(output, 'ct3_' + filename + '.tif'), volume, compress=6)
    #break
    

/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0001.nii.gz
(0.68, 0.68, 8.0)
(512, 512, 43) (348, 348, 344)
['render_with_linear_interp', 'render_with_cubic_interp', 'print_device_params']
render_with_linear_interp
render_with_cubic_interp


Lossy conversion from float32 to uint8. Range [0.0, 78.70359802246094]. Convert image to uint8 prior to saving to suppress this warning.


/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0002.nii.gz
(0.709, 0.709, 8.0)
(512, 512, 42) (363, 363, 336)
['render_with_linear_interp', 'render_with_cubic_interp', 'print_device_params']
render_with_cubic_interp
render_with_cubic_interp


Lossy conversion from float32 to uint8. Range [0.0, 76.39685821533203]. Convert image to uint8 prior to saving to suppress this warning.


/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0003.nii.gz
(0.74, 0.74, 8.0)
(512, 512, 41) (379, 379, 328)
['render_with_linear_interp', 'render_with_cubic_interp', 'print_device_params']
render_with_cubic_interp
render_with_cubic_interp


...

/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0012.nii.gz
(0.972, 0.972, 8.0)
(512, 512, 43) (498, 498, 344)
['render_with_linear_interp', 'render_with_cubic_interp', 'print_device_params']
render_with_cubic_interp
render_with_cubic_interp


Lossy conversion from float32 to uint8. Range [1.9862730904485204e-10, 79.25421905517578]. Convert image to uint8 prior to saving to suppress this warning.


/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0013.nii.gz
(0.74, 0.74, 8.0)
(512, 512, 41) (379, 379, 328)
['render_with_linear_interp', 'render_with_cubic_interp', 'print_device_params']
render_with_cubic_interp
render_with_cubic_interp


Lossy conversion from float32 to uint8. Range [0.0, 70.39569854736328]. Convert image to uint8 prior to saving to suppress this warning.


/home/tmquan/iXrayCT/download/segmentation/mosmed/studies/CT-0/study_0014.nii.gz
(0.741, 0.741, 8.0)
